In [ ]:
# Helpers
def string_similarity(str1, str2):
    """
    Estimates Jaccard distance between str1 and str2
    """
    str1 = str1.strip().lower()
    str2 = str2.strip().lower()
    set1 = set(str1)
    set2 = set(str2)
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    similarity = len(intersection) / len(union)
    return similarity

In [36]:
# Imports
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import etree

# Read df
df = pd.read_excel('input.xlsx')
df = df[['producto','precio']]

# Input values
productos = df['producto'].values.tolist()
precio_compra = df['precio'].values.tolist()
productos_buscable = ['-'.join(p.strip().split(' ')) for p in productos]


# Obtencion del precio en Mercado libre

base_url = 'https://listado.mercadolibre.com.ar/'

lista_precios = []
fallados = []

for i, prod in enumerate(productos_buscable[:100]):
  r = requests.get(base_url+prod)
   
  if r.status_code == 200:
    soup = BeautifulSoup(r.content, 'html.parser')
    
    # Nombres
    nombres = soup.find_all('h2', {'class':'ui-search-item__title shops__item-title'})
    nombres = [n.text for n in nombres]
    
    # Precios
    precios = soup.find_all('span', {'class':'price-tag-fraction'})
    precios = [int(i.text.replace('.','')) for i in precios]

    # Distancia de Jaccard
    precios_correctos = [] 
    for i2, n in enumerate(nombres):
        if string_similarity(productos[i], n) > 0.5:
            precios_correctos.append(precios[i2])
            
    
    if len(precios_correctos)>0:
      lista_precios.append(int(np.mean(precios_correctos)))
      print(f'Encontrado artículo {i+1} de {len(productos_buscable)}\n')
    else:
        print(f'NO encontrado articulo {i} de {len(productos_buscable)}\n')
        fallados.append(productos[i])
        del productos[i]
        del precio_compra[i]
        
  else:
    print(f'Error de request articulo {i} de {len(productos_buscable)}\n')
    fallados.append(productos[i])
    del productos[i]
    del precio_compra[i]
    

# df
ganancia = [lista_precios[i] - precio_compra[i] for i in range(len(lista_precios))]
df_final = pd.DataFrame([productos, ganancia, precio_compra, lista_precios]).T
df_final.columns = ['producto', 'ganancia', 'precio de compra', 'precio de venta estimado']
df_final.sort_values('ganancia', ascending=False, inplace=True)
df_final = df_final.iloc[:len(lista_precios)]

Encontrado artículo 1 de 902

Encontrado artículo 2 de 902

Encontrado artículo 3 de 902

Encontrado artículo 4 de 902

Encontrado artículo 5 de 902

Encontrado artículo 6 de 902

Encontrado artículo 7 de 902

Encontrado artículo 8 de 902

Encontrado artículo 9 de 902

NO encontrado articulo 9 de 902

NO encontrado articulo 10 de 902

Encontrado artículo 12 de 902

NO encontrado articulo 12 de 902

Encontrado artículo 14 de 902

Encontrado artículo 15 de 902

Encontrado artículo 16 de 902

NO encontrado articulo 16 de 902

NO encontrado articulo 17 de 902

NO encontrado articulo 18 de 902

NO encontrado articulo 19 de 902

Encontrado artículo 21 de 902

NO encontrado articulo 21 de 902

Encontrado artículo 23 de 902

Encontrado artículo 24 de 902

NO encontrado articulo 24 de 902

Encontrado artículo 26 de 902

NO encontrado articulo 26 de 902

NO encontrado articulo 27 de 902

NO encontrado articulo 28 de 902

Encontrado artículo 30 de 902

NO encontrado articulo 30 de 902

Encontrado